In [1]:
import sys
sys.path.append('../')

import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import time
import json
import argparse

import tensorflow as tf
import numpy as np
import cv2

from eval import evaluation_model_by_mAP

from tensorflow.keras.layers import Lambda
from yolov3.models import YoloV3, yolo_boxes
from yolov3.anchors import default_anchor

anchors = default_anchor/416

In [14]:
def yolo_nms(outputs, classes, nms_config):    
    # boxes, conf, type
    b, c, t = [], [], []
    
    max_boxes, iou_threshold, score_threshold = nms_config


    for o in outputs:
        b.append(tf.reshape(o[0], (tf.shape(o[0])[0], -1, tf.shape(o[0])[-1])))
        c.append(tf.reshape(o[1], (tf.shape(o[1])[0], -1, tf.shape(o[1])[-1])))
        t.append(tf.reshape(o[2], (tf.shape(o[2])[0], -1, tf.shape(o[2])[-1])))

    bbox = tf.concat(b, axis=1)
    confidence = tf.concat(c, axis=1)
    class_probs = tf.concat(t, axis=1)

    scores = confidence * class_probs

    dscores = tf.squeeze(scores, axis=0)
    scores = tf.reduce_max(dscores,[1])
    bbox = tf.reshape(bbox,(-1,4))
    classes = tf.argmax(dscores,1)
    
    selected_indices, selected_scores = tf.image.non_max_suppression_with_scores(
        boxes=bbox,
        scores=scores,
        max_output_size=max_boxes,
        iou_threshold=iou_threshold,
        score_threshold=score_threshold,
        soft_nms_sigma=0.5
    )
    
    num_valid_nms_boxes = tf.shape(selected_indices)[0]

    selected_indices = tf.concat([selected_indices,tf.zeros(max_boxes-num_valid_nms_boxes, tf.int32)], 0)
    selected_scores = tf.concat([selected_scores,tf.zeros(max_boxes-num_valid_nms_boxes,tf.float32)], -1)

    boxes=tf.gather(bbox, selected_indices)
    boxes = tf.expand_dims(boxes, axis=0)
    scores=selected_scores
    scores = tf.expand_dims(scores, axis=0)
    classes = tf.gather(classes,selected_indices)
    classes = tf.expand_dims(classes, axis=0)
    valid_detections=num_valid_nms_boxes
    valid_detections = tf.expand_dims(valid_detections, axis=0)

    return boxes, scores, classes, valid_detections

In [3]:
# # 평가할 모델 및 config 로드
# config_path = '../configs/fl_test_grid_anchor_default.json'
# with open(config_path,'r') as f: config = json.load(f)
    
# anchors = default_anchor/416
# classname_path = os.path.join(config['data']['root_path'],config['data']['classes_names'])

# # evaluation model 및 weight 로드
# eval_model = YoloV3(config, anchors, classes=config['data']['n_classes'],training=False)
# eval_model_t = YoloV3(config, anchors, classes=config['data']['n_classes'],training=True)

# weight_path = os.path.join('../',config['training']['save_path'], config['model_name'],'trained_mAP_best.tf')
# eval_model.load_weights(weight_path).expect_partial()

In [4]:
# eval_model.save('./2')

In [5]:
model_loaded = tf.saved_model.load('./2')

In [17]:
from yolov3.dataset import transform_images

img_batch = np.zeros((2,800,800,3))/255.
img_batch = transform_images(img_batch,800)

b_0, b_1, b_2 = model_loaded(img_batch)

In [21]:
n_batch = b_0[0].shape[0]

In [ ]:
output_list = []

for i in range(n_batch)
    b0 = b_0[0]
    b1 = b_1[1]
    b2 = b_2[2]
    
    outputs = (lambda x: yolo_nms(x, 5, (300, 0.5, 0.5)))((b_0[:3], b_1[:3], b_2[:3]))

In [15]:
outputs = (lambda x: yolo_nms(x, 5, (300, 0.5, 0.5)))((b_0[:3], b_1[:3], b_2[:3]))